In [32]:
try: 
    import cv2
    import torch
    import torchvision
    import sklearn.svm
except:
    %pip install opencv-python-headless==4.9.0.80
    %pip install torch
    %pip install torchvision
    %pip install torchsummary 
    %pip install sklearn

from torch.utils.tensorboard import SummaryWriter
import torch
from torch.utils.data import Dataset
from torch import cuda
from torchvision import transforms, datasets, models
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

from pathlib import Path
from timeit import default_timer as timer
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import random
import numpy as np
import time
import copy

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


from torchsummary import summary
from PIL import Image

np.set_printoptions(threshold=np.inf)

print('import successful')

torch.cuda.empty_cache()
torch.cuda.is_available()

import successful


True

In [33]:
# Data paths
EMOREACT = Path('EmoReact')
FER = Path('FER-2013')
KDEF = Path('KDEF-AKDEF')
NIMH = Path('NIMH-CHEFS')

# General paths
BASE_PATH = Path('/home/jovyan/work/data/out')
MODEL_PATH = Path('/home/jovyan/work/models')

# Set dataset here
DATA = NIMH

# Dataset-specific paths
CURRENT_PATH = BASE_PATH / DATA
LABELS = [f.name for f in CURRENT_PATH.iterdir() if f.is_dir()]
IMAGE_PATHS = list(CURRENT_PATH.rglob('*.jpg'))

# Constants for splitting dataset
TRAIN = 'train'
TEST = 'test'
VAL = 'val'

FEATURES = 'feature-extraction'
TRANSFER = 'transfer-learning'
FINETUNE = 'fine-tuning'

# batch size
BATCH_SIZE = 10

# CUDA
train_on_gpu = cuda.is_available()
print(f'[INFO] Train on gpu ...{train_on_gpu}')
if train_on_gpu:
    gpu_count = cuda.DEVICE_count()
    print(f'[INFO] {gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False
        
DEVICE = torch.DEVICE("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

[INFO] Train on gpu ...True


AttributeError: module 'torch.cuda' has no attribute 'DEVICE_count'

In [5]:
class Dataset(Dataset):
    def __init__(self, data_path, img_size, transforms=None, phase='train'):
        self.data_path = Path(data_path) / phase
        self.img_size = img_size
        self.transform = transforms[phase]
        self.phase = phase

        self.classes = self._get_classes()
        self.image_paths = self._get_image_paths()
        self.num_classes = len(self.classes)

        self.class_to_int = {class_name: idx for idx, class_name in enumerate(self.classes)}
        self.int_to_class = {idx: class_name for class_name, idx in self.class_to_int.items()}

    def _get_classes(self):
        return [f.name for f in self.data_path.iterdir() if f.is_dir()]

    def _get_image_paths(self):
        paths = list(self.data_path.rglob('*.jpg'))
        random.shuffle(paths)
        return paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx])
        img_path = self.image_paths[idx]

        if self.transform:
            img = self.transform(img)

        label = Path(img_path).parent.name
        label = self.class_to_int[label]  # Convert label to integer

        return img, label

    def show_samples(self):
        fig = plt.figure(figsize=(20, 20))

        for i in range(10):
            ax = fig.add_subplot(1, 10, i + 1)
            _, label = self.__getitem__(i)
            img_cv2 = self.get_cv2_img(i)

            ax.imshow(img_cv2, cmap='gray')
            ax.set_title(self.int_to_class[label])  # Show class name instead of index
            ax.axis('off')
        plt.show()

    def show_distribution(self):
        labels_count = Counter([self.__getitem__(i)[1] for i in range(len(self.image_paths))])
        sorted_counts = sorted(labels_count.items())
        labels, counts = zip(*sorted_counts)

        plt.figure(figsize=(10, 3))
        bars = plt.bar(labels, counts, color='skyblue')
        plt.xlabel('Class')
        plt.ylabel('Count')
        plt.title('Counts per Class')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)

        for bar, count in zip(bars, counts):
            plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5, count,
                    ha='center', va='bottom', color='black', fontsize=8)

        plt.tight_layout()
        plt.show()

    def get_cv2_img(self, idx):
        img_path = self.image_paths[idx]
        return cv2.imread(str(img_path))

    def idx_to_class(self, idx_list):
        return [self.int_to_class[idx] for idx in idx_list]

    def class_to_idx(self, class_list):
        return [self.class_to_int[class_name] for class_name in class_list]

    def print_info(self):
        print(f"[INFO] Total number of images: {len(self)}")
        print("[INFO] Number of classes:", self.num_classes)
        print("[INFO] Classes:", self.classes)


In [6]:
data_transforms = {
    TRAIN: transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

datasets = { x: Dataset(CURRENT_PATH, img_size=224, transforms=data_transforms, phase=x) for x in [TRAIN, VAL, TEST] }
dataloaders = { x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in [TRAIN, VAL, TEST] }
dataset_sizes = { x : len(datasets[x]) for x in [TRAIN, VAL, TEST] }

N_CLASSES = datasets[TRAIN].num_classes

In [11]:
# init the VGG model
vgg19 = models.vgg19(weights='IMAGENET1K_V1')
vgg19 = vgg19.to(DEVICE)

# freeze parameters for feature extraction
for param in vgg19.features.parameters():
    param.require_grad = False

# number of input features for last layer
num_features = vgg19.classifier[6].in_features

vgg19.classifier = torch.nn.Identity()
vgg19.classifier = vgg19.classifier.to(DEVICE)

## Feature Extraction

In [14]:
def extract_features(loader, conv_base):
    conv_base.eval()
    features = []
    labels = []

    with torch.no_grad():
        for images, targets in loader:
            # move data to cuda
            images = images.to(DEVICE)
            targets = torch.as_tensor(targets).to(DEVICE)
            
            # Extract features using conv_base
            features_batch = conv_base(images)
            features.append(features_batch.cpu().numpy())  # Convert to numpy array
            labels.append(targets.cpu().numpy())

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

In [15]:
train_features, train_labels = extract_features(loader=dataloaders[TRAIN], conv_base=vgg19)
val_features, val_labels = extract_features(loader=dataloaders[VAL], conv_base=vgg19)
test_features, test_labels = extract_features(loader=dataloaders[TEST], conv_base=vgg19)

## Training

In [17]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    
    # freeze feature layers
    for param in vgg.features.parameters():
        param.requires_grad = False
    
    # make classifier trainable
    for param in vgg.classifier.parameters():
        param.requires_grad = True
    """
    for param in vgg.features.parameters():
        param.requires_grad = True
    """
    vgg.classifier[-1] = torch.nn.Linear(in_features=vgg.classifier[-1].in_features, out_features=N_CLASSES)
    vgg = vgg.to(DEVICE)

    best_acc = 0.0
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0

    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        print('-' * 10)

        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)

        # iterate through batches of training set
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print(f"\rTraining batch {i}/{train_batches}", flush=True)
            
            # set input and labels to the batch features and labels
            inputs = data[0]
            labels = data[1]

            # move to cuda if possible
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            # do the learning stuff & update loss etc.
            optimizer_ft.zero_grad()
            outputs = vgg(inputs)
            _, preds = torch.max(outputs.data, 1)
            # backward & optimize
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_ft.step()                
            
            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        avg_loss = loss_train / dataset_sizes[TRAIN]
        avg_acc = acc_train / dataset_sizes[TRAIN]

        # set to evaluation mode
        vgg.train(False)
        vgg.eval()

        # iterate through batches of validation set
        with torch.no_grad():
            for i, data in enumerate(dataloaders[VAL]):
                if i % 100 == 0:
                    print(f"\rValidation batch {i}/{val_batches}", flush=True)
                
                # set input and labels to the batch features and labels
                inputs = data[0]
                labels = data[1]
    
                # move to cuda if possible
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
    
                # do the learning stuff & update loss etc.
                # zero the gradients
                optimizer_ft.zero_grad()
                outputs = vgg(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
                loss_val += loss.item()
                acc_val += torch.sum(preds == labels.data)
                
            # delete batch from cuda 
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]

        # print results
        print()
        print(f"Epoch {epoch} result: ".format(epoch))
        print(f"Avg loss (train): {avg_loss:.4f}")
        print(f"Avg acc (train): {avg_acc:.4f}")
        print(f"Avg loss (val): {avg_loss_val:.4f}")
        print(f"Avg acc (val): {avg_acc_val:.4f}")
        print('-' * 10)
        print()

        # update best acc save best model weights
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())

    # compute training time
    elapsed_time = time.time() - since

    # print all the results
    print()
    print(f"Training completed in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s")
    print(f"Best acc: {best_acc:.4f}")

    return vgg.load_state_dict(best_model_wts)

In [19]:
# init the VGG model
vgg19 = models.vgg19(weights='IMAGENET1K_V1')
vgg19 = vgg19.to(DEVICE)

summary(vgg19, input_size=(3, 224, 224), batch_size=BATCH_SIZE, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [10, 64, 224, 224]           1,792
              ReLU-2         [10, 64, 224, 224]               0
            Conv2d-3         [10, 64, 224, 224]          36,928
              ReLU-4         [10, 64, 224, 224]               0
         MaxPool2d-5         [10, 64, 112, 112]               0
            Conv2d-6        [10, 128, 112, 112]          73,856
              ReLU-7        [10, 128, 112, 112]               0
            Conv2d-8        [10, 128, 112, 112]         147,584
              ReLU-9        [10, 128, 112, 112]               0
        MaxPool2d-10          [10, 128, 56, 56]               0
           Conv2d-11          [10, 256, 56, 56]         295,168
             ReLU-12          [10, 256, 56, 56]               0
           Conv2d-13          [10, 256, 56, 56]         590,080
             ReLU-14          [10, 256,

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg19.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [21]:
vgg19 = train_model(vgg19, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 0/50
----------
Training batch 0/32

Validation batch 0/11

Epoch 0 result: 
Avg loss (train): 0.1646
Avg acc (train): 0.2138
Avg loss (val): 0.1681
Avg acc (val): 0.2621
----------

Epoch 1/50
----------
Training batch 0/32

Validation batch 0/11

Epoch 1 result: 
Avg loss (train): 0.1570
Avg acc (train): 0.2893
Avg loss (val): 0.1570
Avg acc (val): 0.4951
----------

Epoch 2/50
----------
Training batch 0/32

Validation batch 0/11

Epoch 2 result: 
Avg loss (train): 0.1510
Avg acc (train): 0.3428
Avg loss (val): 0.1488
Avg acc (val): 0.5340
----------

Epoch 3/50
----------
Training batch 0/32

Validation batch 0/11

Epoch 3 result: 
Avg loss (train): 0.1427
Avg acc (train): 0.4403
Avg loss (val): 0.1365
Avg acc (val): 0.5922
----------

Epoch 4/50
----------
Training batch 0/32

Validation batch 0/11

Epoch 4 result: 
Avg loss (train): 0.1365
Avg acc (train): 0.4717
Avg loss (val): 0.1306
Avg acc (val): 0.5728
----------

Epoch 5/50
----------
Training batch 0/32

Validation b

In [25]:
torch.save(state_dict(), 'vgg19_NIMH-CHEFS.pt')

NameError: name 'state_dict' is not defined

In [26]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print(f"\rTest batch {i}/{test_batches}", flush=True)

        vgg.train(False)
        vgg.eval()
        inputs = data[0]
        labels = data[1]

        # move to cuda if possible
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        # Ddsable gradient calculation for evaluation
        with torch.no_grad():  
            outputs = vgg(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        loss_test += loss.item()
        acc_test += torch.sum(preds == labels.data)

        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print(f"Evaluation completed in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s")
    print(f"Avg loss (test): {avg_loss:.4f}")
    print(f"Avg acc (test): {avg_acc:.4f}")
    print('-' * 10)

In [27]:
eval_model(vgg19, criterion)

Evaluating model
----------
Test batch 0/11


AttributeError: '_IncompatibleKeys' object has no attribute 'train'

In [28]:
vgg19.state_dict()

AttributeError: '_IncompatibleKeys' object has no attribute 'state_dict'

In [31]:
print(vgg19)

<All keys matched successfully>
